In [1]:
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"] = "4,"
from pathlib import Path
import json, shutil
from tqdm import tqdm
import numpy as np
np.set_printoptions(suppress=True)

import pandas as pd
from PIL import Image
from pyquaternion import Quaternion
import torch
import sqlite3
from database import COLMAPDatabase
from scene.colmap_loader import qvec2rotmat, rotmat2qvec

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
root_dir = Path("kubric4d/test/scn02902")
out_dir = Path("samples/scn02902_test2")

(out_dir / "images").mkdir(parents=True, exist_ok=True)
(out_dir / "colmap").mkdir(parents=True, exist_ok=True)
(out_dir / "colmap/images").mkdir(parents=True, exist_ok=True)
(out_dir / "colmap/sparse_custom").mkdir(parents=True, exist_ok=True)
(out_dir / "colmap/sparse/0").mkdir(parents=True, exist_ok=True)
(out_dir / "colmap/dense/workspace").mkdir(parents=True, exist_ok=True)

In [3]:
# cameras = sorted(list(root_dir.glob("*.json")), key=lambda x: int(x.stem.split("_v")[-1]))

# cam_path = cameras[0]
# name = cam_path.stem.replace(root_dir.stem, "frames")
# metadata = json.loads(cam_path.read_bytes())
# img_paths = sorted(list(root_dir.glob(f"{name}/rgba_*.png")))

# img_path = img_paths[0]
# t = 0

# qvec = np.array(metadata['camera']['quaternions'][t])
# T = np.array(metadata['camera']['positions'][t])

# c2w = np.eye(4, dtype=np.float32)
# c2w[0:3, 0:3] = qvec2rotmat(qvec)
# c2w[0:3, 3] = T
# c2w[0:3, 1] *= -1.0
# c2w[0:3, 2] *= -1.0
# w2c = np.linalg.inv(c2w)
# R, T = w2c[:3,:3], w2c[:3,3]
# qvec = rotmat2qvec(R)
# print(qvec)

In [5]:
object_images_txt = ""
object_cameras_txt = ""
idx = 0
cameras = sorted(list(root_dir.glob("*.json")), key=lambda x: int(x.stem.split("_v")[-1]))
cameras = [cameras[4]]
for cam_path in cameras:
    name = cam_path.stem.replace(root_dir.stem, "frames")
    metadata = json.loads(cam_path.read_bytes())
    W, H = metadata['scene']['resolution']

    K = np.abs(metadata['camera']['K'])
    K = torch.tensor(K, dtype=torch.float32)
    K[0, :] *= W
    K[1, :] *= H
    focal = K[0,0]
    # focal = 32.0
    princp = K[0,2], K[1,2]
    for img_path in sorted(list(root_dir.glob(f"{name}/rgba_*.png"))):
        new_img_name = f"{name}-{img_path.stem}.png"
        t = int(img_path.stem.split("_")[-1])
        if t % 4 == 0:
            shutil.copy(img_path, out_dir / f"images/{new_img_name}")
        # if t == 0:
            qvec = np.array(metadata['camera']['quaternions'][t])
            T = np.array(metadata['camera']['positions'][t])

            # R = qvec2rotmat(qvec)
            # R[:, 1] *= -1.0
            # R[:, 2] *= -1.0

            # R = -np.transpose(R)
            # R[:,0] *= -1.0
            # T = -T
            # T = -np.matmul(R,T)
            # qvec = rotmat2qvec(np.transpose(R))

            c2w = np.eye(4, dtype=np.float32)
            c2w[0:3, 0:3] = qvec2rotmat(qvec)
            c2w[0:3, 3] = T
            c2w[0:3, 1] *= -1.0
            c2w[0:3, 2] *= -1.0
            w2c = np.linalg.inv(c2w)
            R, T = w2c[:3,:3], w2c[:3,3]
            qvec = rotmat2qvec(R)

            shutil.copy(img_path, out_dir / f"colmap/images/{new_img_name}")
            object_images_txt += f"{idx+1} {' '.join([str(x) for x in qvec])} {' '.join([str(x) for x in T])} 1 {new_img_name}\n\n"
            object_cameras_txt += f"{idx+1} SIMPLE_PINHOLE {W} {H} {focal} {princp[0]} {princp[1]}\n"
            idx += 1

(out_dir / "colmap/sparse_custom/images.txt").write_text(object_images_txt)
(out_dir / "colmap/sparse_custom/cameras.txt").write_text(object_cameras_txt)
(out_dir / "colmap/sparse_custom/points3D.txt").write_text("")

0

In [6]:
!colmap feature_extractor --database_path $out_dir/colmap/database.db --image_path $out_dir/colmap/images \
 --SiftExtraction.max_image_size 4096 --SiftExtraction.max_num_features 16384 \
 --SiftExtraction.estimate_affine_shape 1 \
 --SiftExtraction.domain_size_pooling 1

I0115 21:18:40.194579 161324 misc.cc:44] 
Feature extraction
I0115 21:18:40.259552 161421 sift.cc:717] Creating Covariant SIFT CPU feature extractor
I0115 21:18:40.260349 161422 sift.cc:717] Creating Covariant SIFT CPU feature extractor
I0115 21:18:40.260493 161423 sift.cc:717] Creating Covariant SIFT CPU feature extractor
I0115 21:18:40.261437 161425 sift.cc:717] Creating Covariant SIFT CPU feature extractor
I0115 21:18:40.261885 161426 sift.cc:717] Creating Covariant SIFT CPU feature extractor
I0115 21:18:40.267939 161433 sift.cc:717] Creating Covariant SIFT CPU feature extractor
I0115 21:18:40.263507 161428 sift.cc:717] Creating Covariant SIFT CPU feature extractor
I0115 21:18:40.264101 161424 sift.cc:717] Creating Covariant SIFT CPU feature extractor
I0115 21:18:40.264361 161429 sift.cc:717] Creating Covariant SIFT CPU feature extractor
I0115 21:18:40.265197 161430 sift.cc:717] Creating Covariant SIFT CPU feature extractor
I0115 21:18:40.266053 161431 sift.cc:717] Creating Covarian

In [7]:
camModelDict = {
    'SIMPLE_PINHOLE': 0, 'PINHOLE': 1, 'SIMPLE_RADIAL': 2, 'RADIAL': 3,
    'OPENCV': 4, 'FULL_OPENCV': 5, 'SIMPLE_RADIAL_FISHEYE': 6,
    'RADIAL_FISHEYE': 7, 'OPENCV_FISHEYE': 8, 'FOV': 9,
    'THIN_PRISM_FISHEYE': 10
}

db = COLMAPDatabase.connect(out_dir / "colmap/database.db")
# db = COLMAPDatabase.connect("data/kubric_car/colmap/kubric_car/database.db")

# res = db.execute("SELECT sql FROM sqlite_master WHERE tbl_name = 'cameras' AND type = 'table'")
# element = res.fetchone()[0].replace('(', '').replace(')', '')
# # print("\n".join([x.strip() for x in ' '.join(element.split('cameras')[1:]).strip().split(",   ")]))

# res = db.execute("SELECT * FROM cameras")
# all_els = res.fetchall()
# df = pd.DataFrame(all_els, columns=['camera_id', 'model', 'width', 'height', 'params', 'prior_focal_length'])
# df['params'] = df['params'].apply(lambda blob: np.fromstring(blob, dtype=np.float64).reshape((-1,)))

camera_lines = (out_dir / "colmap/sparse_custom/cameras.txt").read_text()
for line in camera_lines.strip().split("\n"):
    cameraId, cameraModel, width, height, focal, princpx, princpy = line.split(' ')
    cameraModel = camModelDict[cameraModel]
    params = np.array([focal, princpx, princpy]).astype(np.float64)
    camera_id = db.update_camera(cameraModel, int(float(width)), int(float(height)), params, int(cameraId))
db.commit()
# print(int(cameraId), cameraModel, int(float(width)), int(float(height)), params)

res = db.execute("SELECT * FROM cameras")
all_els = res.fetchall()
df = pd.DataFrame(all_els, columns=['camera_id', 'model', 'width', 'height', 'params', 'prior_focal_length'])
df['params'] = df['params'].apply(lambda blob: np.fromstring(blob, dtype=np.float64).reshape((-1,)))
df

,camera_id,model,width,height,params,prior_focal_length
0,1,0,576,384,"[576.0, 288.0, 192.0]",1
1,2,0,576,384,"[576.0, 288.0, 192.0]",1
2,3,0,576,384,"[576.0, 288.0, 192.0]",1
3,4,0,576,384,"[576.0, 288.0, 192.0]",1
4,5,0,576,384,"[576.0, 288.0, 192.0]",1
5,6,0,576,384,"[576.0, 288.0, 192.0]",1
6,7,0,576,384,"[576.0, 288.0, 192.0]",1
7,8,0,576,384,"[576.0, 288.0, 192.0]",1
8,9,0,576,384,"[576.0, 288.0, 192.0]",1
9,10,0,576,384,"[576.0, 288.0, 192.0]",1


In [8]:
!colmap exhaustive_matcher --database_path $out_dir/colmap/database.db --SiftMatching.guided_matching=true
!colmap point_triangulator \
 --database_path $out_dir/colmap/database.db \
 --image_path $out_dir/colmap/images \
 --input_path $out_dir/colmap/sparse_custom \
 --output_path $out_dir/colmap/sparse/0

I0115 21:18:42.372572 161521 misc.cc:44] 
Feature matching
I0115 21:18:42.373462 161522 sift.cc:1426] Creating SIFT GPU feature matcher
I0115 21:18:42.434458 161620 sift.cc:1426] Creating SIFT GPU feature matcher
I0115 21:18:42.954293 161521 pairing.cc:168] Generating exhaustive image pairs...
I0115 21:18:42.954331 161521 pairing.cc:201] Matching block [1/1, 1/1]
I0115 21:18:43.129029 161521 feature_matching.cc:46] in 0.175s
I0115 21:18:43.130620 161521 timer.cc:91] Elapsed time: 0.013 [minutes]
I0115 21:18:43.601706 161621 misc.cc:44] 
Loading model
I0115 21:18:43.603085 161621 incremental_pipeline.cc:237] Loading database
I0115 21:18:43.603701 161621 database_cache.cc:66] Loading cameras...
I0115 21:18:43.603734 161621 database_cache.cc:76]  15 in 0.000s
I0115 21:18:43.603740 161621 database_cache.cc:84] Loading matches...
I0115 21:18:43.605270 161621 database_cache.cc:89]  105 in 0.002s
I0115 21:18:43.605278 161621 database_cache.cc:105] Loading images...
I0115 21:18:43.606105 16162

In [9]:
!colmap image_undistorter --image_path $out_dir/colmap/images --input_path $out_dir/colmap/sparse/0 --output_path $out_dir/colmap/dense/workspace
!colmap patch_match_stereo --workspace_path $out_dir/colmap/dense/workspace
!colmap stereo_fusion --workspace_path $out_dir/colmap/dense/workspace --output_path $out_dir/colmap/dense/workspace/fused.ply
!python scripts/downsample_point.py $out_dir/colmap/dense/workspace/fused.ply $out_dir/colmap/point3D_multipleview.ply

I0115 21:18:57.460438 161622 misc.cc:44] 
Reading reconstruction
I0115 21:18:57.462610 161622 image.cc:347] => Reconstruction with 15 images and 0 points
I0115 21:18:57.462630 161622 misc.cc:44] 
Image undistortion
I0115 21:18:57.602885 161622 undistortion.cc:215] Undistorting image [1/15]
I0115 21:18:57.602907 161634 undistortion.cc:257] Undistorted image found; copying to location: samples/scn02902_test2/colmap/dense/workspace/images/frames_p0_v4-rgba_00048.png
I0115 21:18:57.602898 161631 undistortion.cc:257] Undistorted image found; copying to location: samples/scn02902_test2/colmap/dense/workspace/images/frames_p0_v4-rgba_00016.png
I0115 21:18:57.602898 161625 undistortion.cc:257] Undistorted image found; copying to location: samples/scn02902_test2/colmap/dense/workspace/images/frames_p0_v4-rgba_00004.png
I0115 21:18:57.602900 161623 undistortion.cc:257] Undistorted image found; copying to location: samples/scn02902_test2/colmap/dense/workspace/images/frames_p0_v4-rgba_00000.png
I

In [16]:
from scene.colmap_loader import read_extrinsics_text, read_intrinsics_text, qvec2rotmat, \
    read_extrinsics_binary, read_intrinsics_binary, read_points3D_binary, read_points3D_text

datadir = "samples/scn02902/colmap"

cameras_extrinsic_file = os.path.join(datadir, "sparse/0/images.bin")
cameras_intrinsic_file = os.path.join(datadir, "sparse/0/cameras.bin")
cam_extrinsics = read_extrinsics_binary(cameras_extrinsic_file)
cam_intrinsics = read_intrinsics_binary(cameras_intrinsic_file)
cam_folder=os.path.join(datadir, "../")

image_folder = os.path.join(cam_folder, "images")
image_range = list(range(0, 60, 4))

image_paths, image_poses, image_times = [], [], []
for idx, key in enumerate(cam_extrinsics):
    extr = cam_extrinsics[key]
    R = np.transpose(qvec2rotmat(extr.qvec))
    T = np.array(extr.tvec)
    image_path_0 = os.path.join(image_folder, extr.name)
    for time in image_range:
        image_path = os.path.join(
            os.path.dirname(image_path_0), 
            os.path.basename(image_path_0).replace(
                "rgba_00000.png", f"rgba_{time:05d}.png"
            )
        )
        print(image_path)

/data/3D_dataset/MVS/4DGaussians/samples/scn02902/colmap/../images/frames_p0_v10-rgba_00000.png
/data/3D_dataset/MVS/4DGaussians/samples/scn02902/colmap/../images/frames_p0_v10-rgba_00004.png
/data/3D_dataset/MVS/4DGaussians/samples/scn02902/colmap/../images/frames_p0_v10-rgba_00008.png
/data/3D_dataset/MVS/4DGaussians/samples/scn02902/colmap/../images/frames_p0_v10-rgba_00012.png
/data/3D_dataset/MVS/4DGaussians/samples/scn02902/colmap/../images/frames_p0_v10-rgba_00016.png
/data/3D_dataset/MVS/4DGaussians/samples/scn02902/colmap/../images/frames_p0_v10-rgba_00020.png
/data/3D_dataset/MVS/4DGaussians/samples/scn02902/colmap/../images/frames_p0_v10-rgba_00024.png
/data/3D_dataset/MVS/4DGaussians/samples/scn02902/colmap/../images/frames_p0_v10-rgba_00028.png
/data/3D_dataset/MVS/4DGaussians/samples/scn02902/colmap/../images/frames_p0_v10-rgba_00032.png
/data/3D_dataset/MVS/4DGaussians/samples/scn02902/colmap/../images/frames_p0_v10-rgba_00036.png
/data/3D_dataset/MVS/4DGaussians/samples

# ORIGIN

In [2]:
root_dir = Path("data/espresso")
(root_dir / "colmap").mkdir(parents=True, exist_ok=True)
(root_dir / "colmap/images").mkdir(parents=True, exist_ok=True)
(root_dir / "colmap/sparse_custom").mkdir(parents=True, exist_ok=True)
(root_dir / "colmap/sparse/0").mkdir(parents=True, exist_ok=True)
(root_dir / "colmap/dense/workspace").mkdir(parents=True, exist_ok=True)

### hypernerf2colmap

In [ ]:
def rotmat2qvec(R):
    Rxx, Ryx, Rzx, Rxy, Ryy, Rzy, Rxz, Ryz, Rzz = R.flat
    K = np.array([
        [Rxx - Ryy - Rzz, 0, 0, 0],
        [Ryx + Rxy, Ryy - Rxx - Rzz, 0, 0],
        [Rzx + Rxz, Rzy + Ryz, Rzz - Rxx - Ryy, 0],
        [Ryz - Rzy, Rzx - Rxz, Rxy - Ryx, Rxx + Ryy + Rzz]]) / 3.0
    eigvals, eigvecs = np.linalg.eigh(K)
    qvec = eigvecs[[3, 0, 1, 2], np.argmax(eigvals)]
    if qvec[0] < 0:
        qvec *= -1
    return qvec
    
images = sorted(list((root_dir / "rgb/2x").glob("*.png")))
cameras = sorted(list((root_dir / "camera").glob("*.json")))
cams = [json.loads(x.read_bytes()) for x in cameras]
sizes = (len(cams) // 200) + 1

object_images_txt = ""
object_cameras_txt = ""
for cnt, (cam, image) in enumerate(zip(cams, images)):
    if (cnt+1) % sizes == 0:
        idx = cnt // sizes
        R = np.array(cam['orientation']).T
        T = -np.array(cam['position']) @ R 
        qvec = rotmat2qvec(R.T)
        object_images_txt += f"{idx+1} {' '.join([str(x) for x in qvec])} {' '.join([str(x) for x in T])} 1 {image.name}\n\n"

        img_size = cam['image_size'][0]/2, cam['image_size'][1]/2
        focal = cam['focal_length']/2
        princp = cam['principal_point'][0]/2, cam['principal_point'][1]/2
        object_cameras_txt += f"{idx+1} SIMPLE_PINHOLE {img_size[0]} {img_size[1]} {focal} {princp[0]} {princp[1]}\n"

        shutil.copy(image, root_dir / f"colmap/images/{image.name}")

(root_dir / "colmap/sparse_custom/images.txt").write_text(object_images_txt)
(root_dir / "colmap/sparse_custom/cameras.txt").write_text(object_cameras_txt)
(root_dir / "colmap/sparse_custom/points3D.txt").write_text("")

In [ ]:
!colmap feature_extractor --database_path $root_dir/colmap/database.db --image_path $root_dir/colmap/images \
 --SiftExtraction.max_image_size 4096 --SiftExtraction.max_num_features 16384 --SiftExtraction.estimate_affine_shape 1 \
 --SiftExtraction.domain_size_pooling 1

### database

In [ ]:
camModelDict = {
    'SIMPLE_PINHOLE': 0, 'PINHOLE': 1, 'SIMPLE_RADIAL': 2, 'RADIAL': 3,
    'OPENCV': 4, 'FULL_OPENCV': 5, 'SIMPLE_RADIAL_FISHEYE': 6,
    'RADIAL_FISHEYE': 7, 'OPENCV_FISHEYE': 8, 'FOV': 9,
    'THIN_PRISM_FISHEYE': 10
}

class COLMAPDatabase(sqlite3.Connection):
    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)

    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = lambda: self.executescript(CREATE_DESCRIPTORS_TABLE)
        self.create_images_table = lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = lambda: self.executescript(CREATE_TWO_VIEW_GEOMETRIES_TABLE)
        self.create_keypoints_table = lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def update_camera(self, model, width, height, params, camera_id):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "UPDATE cameras SET model=?, width=?, height=?, params=?, prior_focal_length=True WHERE camera_id=?",
            (model, width, height, params.tostring(), camera_id))
        return cursor.lastrowid

db = COLMAPDatabase.connect(root_dir / "colmap/database.db")

res = db.execute("SELECT sql FROM sqlite_master WHERE tbl_name = 'cameras' AND type = 'table'")
element = res.fetchone()[0].replace('(', '').replace(')', '')
print("\n".join([x.strip() for x in ' '.join(element.split('cameras')[1:]).strip().split(",   ")]))

res = db.execute("SELECT * FROM cameras")
all_els = res.fetchall()
df = pd.DataFrame(all_els, columns=['camera_id', 'model', 'width', 'height', 'params', 'prior_focal_length'])
df['params'] = df['params'].apply(lambda blob: np.fromstring(blob, dtype=np.float64).reshape((-1,)))
df

In [ ]:
camera_lines = (root_dir / "colmap/sparse_custom/cameras.txt").read_text()
for line in camera_lines.strip().split("\n"):
    cameraId, cameraModel, width, height, focal, princpx, princpy = line.split(' ')
    cameraModel = camModelDict[cameraModel]
    params = np.array([focal, princpx, princpy]).astype(np.float64)
    camera_id = db.update_camera(cameraModel, int(float(width)), int(float(height)), params, int(cameraId))
db.commit()
# print(int(cameraId), cameraModel, int(float(width)), int(float(height)), params)

res = db.execute("SELECT * FROM cameras")
all_els = res.fetchall()
df = pd.DataFrame(all_els, columns=['camera_id', 'model', 'width', 'height', 'params', 'prior_focal_length'])
df['params'] = df['params'].apply(lambda blob: np.fromstring(blob, dtype=np.float64).reshape((-1,)))
df

In [ ]:
!colmap exhaustive_matcher --database_path $root_dir/colmap/database.db

In [ ]:
!colmap point_triangulator --database_path $root_dir/colmap/database.db --image_path $root_dir/colmap/images \
 --input_path $root_dir/colmap/sparse_custom --output_path $root_dir/colmap/sparse/0 --clear_points 1

In [ ]:
!colmap image_undistorter --image_path $root_dir/colmap/images --input_path $root_dir/colmap/sparse/0 --output_path $root_dir/colmap/dense/workspace

In [ ]:
!colmap patch_match_stereo --workspace_path $root_dir/colmap/dense/workspace

In [ ]:
!colmap stereo_fusion --workspace_path $root_dir/colmap/dense/workspace --output_path $root_dir/colmap/dense/workspace/fused.ply

In [ ]:
!python scripts/downsample_point.py $root_dir/colmap/dense/workspace/fused.ply $root_dir/points3D_downsample2.ply